In [1]:
import cv2
import numpy as np
import math
BODY_PARTS = { "Wrist": 0,
                   "ThumbMetacarpal": 1, "ThumbProximal": 2, "ThumbMiddle": 3, "ThumbDistal": 4,
                   "IndexFingerMetacarpal": 5, "IndexFingerProximal": 6, "IndexFingerMiddle": 7, "IndexFingerDistal": 8,
                   "MiddleFingerMetacarpal": 9, "MiddleFingerProximal": 10, "MiddleFingerMiddle": 11, "MiddleFingerDistal": 12,
                   "RingFingerMetacarpal": 13, "RingFingerProximal": 14, "RingFingerMiddle": 15, "RingFingerDistal": 16,
                   "LittleFingerMetacarpal": 17, "LittleFingerProximal": 18, "LittleFingerMiddle": 19, "LittleFingerDistal": 20,
                 }

POSE_PAIRS = [ ["Wrist", "ThumbMetacarpal"], ["ThumbMetacarpal", "ThumbProximal"],
               ["ThumbProximal", "ThumbMiddle"], ["ThumbMiddle", "ThumbDistal"],
               ["Wrist", "IndexFingerMetacarpal"], ["IndexFingerMetacarpal", "IndexFingerProximal"],
               ["IndexFingerProximal", "IndexFingerMiddle"], ["IndexFingerMiddle", "IndexFingerDistal"],
               ["Wrist", "MiddleFingerMetacarpal"], ["MiddleFingerMetacarpal", "MiddleFingerProximal"],
               ["MiddleFingerProximal", "MiddleFingerMiddle"], ["MiddleFingerMiddle", "MiddleFingerDistal"],
               ["Wrist", "RingFingerMetacarpal"], ["RingFingerMetacarpal", "RingFingerProximal"],
               ["RingFingerProximal", "RingFingerMiddle"], ["RingFingerMiddle", "RingFingerDistal"],
               ["Wrist", "LittleFingerMetacarpal"], ["LittleFingerMetacarpal", "LittleFingerProximal"],
               ["LittleFingerProximal", "LittleFingerMiddle"], ["LittleFingerMiddle", "LittleFingerDistal"] ]
protoFile = "pose_deploy.prototxt"
weightsFile = "pose_iter_102000.caffemodel"
nPoints = 22
inWidth = 368
inHeight = 368
frame = cv2.imread("hand1.jpg")
frameCopy = frame.copy()
frameWidth = frame.shape[1]
frameHeight = frame.shape[0]
threshold = 0.1
net = cv2.dnn.readNetFromCaffe(protoFile, weightsFile)
#Get predictions
inpBlob = cv2.dnn.blobFromImage(frame, 1.0 / 255, (inWidth, inHeight),(0, 0, 0), swapRB=False, crop=False)
net.setInput(inpBlob)
out = net.forward()
#Shoe detections
points = []
'''
for i in range(nPoints):
    # confidence map of corresponding body's part.
    probMap = output[0, i, :, :]
    probMap = cv2.resize(probMap, (frameWidth, frameHeight))

    # Find global maxima of the probMap.
    minVal, prob, minLoc, point = cv2.minMaxLoc(probMap)

    if prob > threshold :
        cv2.circle(frameCopy, (int(point[0]), int(point[1])), 8, (0, 255, 255), thickness=-1, lineType=cv2.FILLED)
        cv2.putText(frameCopy, "{}".format(i), (int(point[0]), int(point[1])), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, lineType=cv2.LINE_AA)

        # Add the point to the list if the probability is greater than the threshold
        points.append((int(point[0]), int(point[1])))
    else :
        points.append(None)
        #cv2.imshow('Output-Keypoints', frameCopy)
# Draw Skeleton
for pair in POSE_PAIRS:
    partA = pair[0]
    partB = pair[1]

    if points[partA] and points[partB]:
        cv2.line(frame, points[partA], points[partB], (0, 255, 255), 2)
        cv2.circle(frame, points[partA], 8, (0, 0, 255), thickness=-1, lineType=cv2.FILLED)
cv2.imshow('Output-Skeleton', frame)
    #if cv2.waitKey(3000) & 0xFF == ord('q'):
        #break

#cap.release() 
cv2.waitKey(0)
cv2.destroyAllWindows()
'''
for i in range(len(BODY_PARTS)):
    # Slice heatmap of corresponging body's part.
    heatMap = out[0, i, :, :]

    # Originally, we try to find all the local maximums. To simplify a sample
    # we just find a global one. However only a single pose at the same time
    # could be detected this way.
    _, conf, _, point = cv2.minMaxLoc(heatMap)
    x = (frameWidth * point[0]) / out.shape[3]
    y = (frameHeight * point[1]) / out.shape[2]
    # Add a point if it's confidence is higher than threshold.
    points.append((int(x), int(y)) if conf > 0.1 else None)
    #print(point[0],point[1])
print(points)
for pair in POSE_PAIRS:
    partFrom = pair[0]
    partTo = pair[1]
    assert(partFrom in BODY_PARTS)
    assert(partTo in BODY_PARTS)

    idFrom = BODY_PARTS[partFrom]
    idTo = BODY_PARTS[partTo]
    #print(idFrom,idTo)
    if points[idFrom] and points[idTo]:
        cv2.line(frame, points[idFrom], points[idTo], (0, 255, 0), 3)
        cv2.ellipse(frame, points[idFrom], (3, 3), 0, 0, 360, (0, 0, 255), cv2.FILLED)
        cv2.ellipse(frame, points[idTo], (3, 3), 0, 0, 360, (0, 0, 255), cv2.FILLED)
        #D = dist.euclidean((points[idFrom][0],points[idTo][0]), (points[idFrom][1],points[idTo][1]))
        distance = math.sqrt( ((points[idFrom][0]-points[idTo][0])**2)+((points[idFrom][1]-points[idTo][1])**2) )

        #print(distance)
distance1 = math.sqrt( ((points[2][0]-points[5][0])**2)+((points[2][1]-points[5][1])**2) )
#print(distance1)
t, _ = net.getPerfProfile()
freq = cv2.getTickFrequency() / 1000
cv2.putText(frame, '%.2fms' % (t / freq), (10, 20), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0))

cv2.imshow('OpenPose using OpenCV', frame)
cv2.waitKey()
cv2.destroyAllWindows()

[(309, 685), (241, 625), (174, 524), (134, 484), (80, 423), (282, 464), (268, 363), (282, 302), (282, 242), (335, 464), (335, 342), (349, 282), (362, 221), (362, 484), (403, 383), (416, 322), (429, 262), (403, 504), (456, 464), (483, 403), (510, 383)]


In [6]:
points[5:9]

[(282, 464), (268, 363), (282, 302), (282, 242)]

In [8]:
points[9:13]

[(335, 464), (335, 342), (349, 282), (362, 221)]

In [26]:
hring = cv2.imread('hring1.jpg')
hring.shape

(84, 103, 3)

In [17]:
from scipy import ndimage

#rotation angle in degree


In [49]:
while True:
    image1 = cv2.imread('hring.jpg')
    #image1 = ndimage.rotate(image1, -30)
    rows = image1.shape[0]
    cols = image1.shape[1]
    img_center = (cols / 2, rows / 2)
    M = cv2.getRotationMatrix2D(img_center, -40, 1)
    image1 = cv2.warpAffine(image1, M, (cols, rows),borderMode=cv2.BORDER_CONSTANT,borderValue=(255,255,255))
    print(image1.shape[0])
    dim = (int(image1.shape[1]),int(image1.shape[0]))
    print(dim)
    img2gray = cv2.cvtColor(image1, cv2.COLOR_BGR2GRAY)

    # add a threshold
    ret, mask = cv2.threshold(img2gray, 220, 255, cv2.THRESH_BINARY_INV)
    #cv2.imshow('Ear Detector', ear_img) 
    mask_inv = cv2.bitwise_not(mask)
    resized = cv2.resize(image1,dim, interpolation = cv2.INTER_AREA)
    print(resized.shape)
    mask = cv2.resize(mask, (resized.shape[1],resized.shape[0]), interpolation = cv2.INTER_AREA)
    print(mask.shape)
    mask_inv = cv2.resize(mask_inv, (resized.shape[1],resized.shape[0]), interpolation = cv2.INTER_AREA)
    print(mask_inv.shape)

    roi = frame[350:350+dim[1],300:300+dim[0]]
    #print(s1,point,point1,point1+dim[0])
    print(roi.shape)

    #print(point,point1)
    roi_bg = cv2.bitwise_and(roi,roi,mask = mask_inv)
    print(roi_bg.shape)
    # roi_fg contains the orignal location of earring
    roi_fg = cv2.bitwise_and(resized,resized,mask = mask)
    print(roi_bg.shape)
    # joining the roi_bg and roi_fg
    dst = cv2.add(roi_bg,roi_fg)
    print(dst.shape)
    #cv2.imshow('Ear Detector', dst) 
    #cv2.waitKey(10)
    #point = int(s1+dim[0])
    #print(point,point+dim[0])
    #cv2.circle(image, (s1,s2+80), 3, (0, 255, 0), -1)

    #cv2.rectangle(image, (s1,s2+80), (s1+dim[0],s2+80+dim[1]), (0,255,0), 3)
    frame[350:350+dim[1],300:300+dim[0]]=dst
    #cv2.imshow('resize traget',roi )
    #cv2.imshow('resize traget',resized )
    cv2.imshow('Ear Detector', frame)
    #cv2.imshow('Ear Detector', dst)

    #c = cv2.waitKey(1) 
    if cv2.waitKey(0) & 0xFF == ord('q'):
            break

#cap.release() 
cv2.destroyAllWindows()

56
(76, 56)
(56, 76, 3)
(56, 76)
(56, 76)
(56, 76, 3)
(56, 76, 3)
(56, 76, 3)
(56, 76, 3)
56
(76, 56)
(56, 76, 3)
(56, 76)
(56, 76)
(56, 76, 3)
(56, 76, 3)
(56, 76, 3)
(56, 76, 3)


In [39]:
rotated_image = cv2.warpAffine(image1, M, (cols, rows),
                           borderMode=cv2.BORDER_CONSTANT,
                           borderValue=(0,255,0))

ValueError: bad transparency mask